In [1]:
%%capture
%run 3-tfidf-gensim.ipynb

In [2]:
from gensim.models import LdaModel
# %pip install pyldavis==3.3.1
import pyLDAvis
import pyLDAvis.gensim

The LDA model uses Dirichlet distributions. The parameters of Dirichlet distributions can be either symmetrical, in which case all the values of the variable will have equal weight, or asymmetrical, where some values have higher weight than others. Moreover, the parameters can be equal to, smaller or greater than 1. For parameters equal to 1, the distribution will assign uniform probability. For values greater than 1, more probability will be assigned to the center, while for values smaller than 1 more probability will be assigned to the corners.

In the LDA model, alpha refers to the Dirichlet distribution that assigns documents to topics. For alpha asymmetric, we might risk of having topics more probable than others. For alpha smaller than 1, we are assuming the documents consist of a small number of topics, while for alpha greater than we assume the documents can be composed of more topics. In our case, it'd be better to have a symmetrical alpha, since we don't know if there's a topic much more likely than others.

For eta we apply the same logic but with topics and words. Eta smaller than 1 means we're assuming topics are not composed of many words, while Eta greater than 1 we assume more words constitute topics. In this case, it's better to let the model find the proper settings.

In [3]:
# Fit an LDA model
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=20,
                     alpha='symmetric', eta='auto', passes=10, random_state=1)

auto_alpha = lda_model.alpha
auto_eta = lda_model.eta

print(f"Automatically determined alpha: {auto_alpha}")
print(f"Automatically determined eta: {auto_eta}")

Automatically determined alpha: [0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05]
Automatically determined eta: [0.05121099 0.04854251 0.0518571  ... 0.04880466 0.04619464 0.04864057]


In [5]:
# Visualize the LDA model using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='tsne')
pyLDAvis.display(vis)

In [6]:
lda_model[corpus]

# it returns a list of lists, where each inner list contains tuples where the topic ID and the topic probability are contained
# Each inner list is associated to a specific document

# output example
''' [
  DOC1: [(0, 0.05), (1, 0.09), ...]
  DOC2: [(0, 0.1), ...]
  .
  .
  .  
]
'''

' [\n  DOC1: [(0, 0.05), (1, 0.09), ...]\n  DOC2: [(0, 0.1), ...]\n  .\n  .\n  .  \n]\n'

In [7]:
# this function returns the top words for each topic by highest frequency
lda_model.show_topic(0, topn=10)

[('clipper', 0.057787415),
 ('terrorism', 0.037640557),
 ('skepticism', 0.0191904),
 ('defense', 0.015877327),
 ('jordan', 0.015788404),
 ('bomb', 0.015692145),
 ('attack', 0.015137033),
 ('brother', 0.013746186),
 ('draft', 0.013724028),
 ('pool', 0.012728827)]

In [8]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df['Clean_Content']):
    # Init output
    sent_topics_df = pd.DataFrame(columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        # we want to sort the tuples in the list 'row' by descending order of the second element, which is the topic probability
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic, i.e. tuple with the highest topic probability
                wp = ldamodel.show_topic(topic_num, topn = 5)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df.loc[len(sent_topics_df)] = [int(topic_num), round(prop_topic, 4), topic_keywords]
            else:
                break
    # Add original text to the end of the output
    sent_topics_df = pd.concat([sent_topics_df, texts], axis=1)
    return sent_topics_df

df_topic_sents_keywords = format_topics_sentences()


In [9]:
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Clean_Content
0,3,0.5690,"time, way, government, world, god",atheist music atheism music fiction distributi...
1,3,0.6960,"time, way, government, world, god",introduction file atheism distribution world o...
2,3,0.5357,"time, way, government, world, god",gospel organization theology evidence luke con...
3,3,0.4124,"time, way, government, world, god",separation organization mantis become none fly...
4,3,0.5742,"time, way, government, world, god",strom al ax fund boy distribution organization...
...,...,...,...,...
11309,3,0.4831,"time, way, government, world, god",promise organization buffalo god peace
11310,3,0.5137,"time, way, government, world, god",promise organization tourist bureau barney spe...
11311,3,0.2741,"time, way, government, world, god",protestant organization tourist bureau concept...
11312,3,0.5170,"time, way, government, world, god",cutter backing tape distribution world organiz...


In [10]:
df_topic_sents_keywords['Dominant_Topic'].unique()

array([ 3,  8,  5, 10, 17, 14, 12, 15, 11,  6, 18,  9, 19, 13,  2, 16,  0,
        1,  7,  4], dtype=int64)

In [11]:
df_topic_distribution = df_topic_sents_keywords.groupby(['Dominant_Topic', 'Topic_Keywords']).size().sort_values(ascending=False).reset_index(name='count')

df_topic_distribution

,Dominant_Topic,Topic_Keywords,count
0,3,"time, way, government, world, god",7521
1,14,"drive, card, sale, bike, computer",2177
2,17,"york, theory, party, compound, space",437
3,5,"window, file, program, server, staff",385
4,10,"pat, printer, color, cancer, express",278
5,12,"security, apple, key, jet, propulsion",99
6,13,"gun, lord, crime, son, minority",56
7,19,"drug, trade, pope, river, york",55
8,15,"president, blood, season, congress, league",49
9,7,"team, health, island, town, josh",38
